##  This notebook allows the user to select XML collections and zip them up to send to a service that runs a transform on them and returns a simple CSV made up of six data points. The data included is the Collection name, Dialect name, Record name, Concept name, Content, Xpath location, and the Dialect Definition for the concept. 

## This CSV contains a row for each concept that is found, so some locations may fulfill multiple concepts. A good example of this are the cncepts Keyword and Place Keyword. Every Place Keyword is also a Keyword, so the row would repeat with a different Concept name. It also contains a row for each undefined node that contains text, marking these rows with an Unknown in the Concept column. 

## This data can be used in a variety of analyses including RAD and QuickE as well as Concept Verticals. It can also be used to teach the system dialect definitions for concepts that are currently unknown by exposing all of the content at undefined nodes. 

In [20]:
%%HTML
<img src=https://image.slidesharecdn.com/scgordonesipwinter2017-170125170939/95/recommendations-analysis-dashboard-1-1024.jpg>

In [21]:
import pandas as pd
import os
from os import walk
import shutil
from ipywidgets import *
import ipywidgets as widgets

In [22]:
Organizations = []
for (dirpath, dirnames, filenames) in walk('../collection/'):
    Organizations.extend(dirnames)
    break  

In [23]:
def OrganizationChoices(organization):
    global OrganizationChoice
    global Organization
    Organization=organization
    print("Organization of the collection is", Organization)


In [24]:
interactive(OrganizationChoices, organization=Organizations)

Organization of the collection is BCO-DMO


In [25]:
Collections = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization)):
    Collections.extend(dirnames)
    break 
Collections

['GeoTraces']

In [26]:
def CollectionChoices(collection):
    global CollectionChoice
    global Collection
    Collection=collection

In [27]:
interactive(CollectionChoices, collection=Collections)

In [28]:
Dialects = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization,Collection)):
    Dialects.extend(dirnames)
    break 
dialectList=Dialects


In [29]:
def dialectChoice(dialect):
    global Dialect
    Dialect=dialect
    print("Dialect of the collection is", Dialect)


In [30]:
interactive(dialectChoice,dialect=dialectList)

Dialect of the collection is ISO


In [31]:
cd ../zip

/Users/scgordon/MILE2/zip


In [32]:
MetadataDestination=os.path.join(Organization,Collection,Dialect,'xml')
MetadataDestination

'BCO-DMO/GeoTraces/ISO/xml'

In [33]:
os.makedirs(MetadataDestination, exist_ok=True)

In [34]:
MetadataLocation=os.path.join('../collection/',Organization,Collection,Dialect,'xml')

MetadataLocation

'../collection/BCO-DMO/GeoTraces/ISO/xml'

In [35]:
src_files = os.listdir(MetadataLocation)
for file_name in src_files:
    full_file_name = os.path.join(MetadataLocation, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, MetadataDestination)

In [36]:
shutil.make_archive('../upload/metadata', 'zip', os.getcwd())

'/Users/scgordon/MILE2/upload/metadata.zip'

In [37]:
cd ../upload

/Users/scgordon/MILE2/upload


In [40]:
%%bash
curl -o ../data/data.csv -F "zipxml=@metadata.zip" http://metadig.nceas.ucsb.edu/metadata/evaluator

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.2M  100 25.2M  100 2000k   414k  32809  0:01:02  0:01:02 --:--:--  499k


In [41]:
%cd ../
shutil.rmtree('upload')
%cd zip
shutil.rmtree(Organization)

/Users/scgordon/MILE2
/Users/scgordon/MILE2/zip


### Now that we have our metadata data prepared, we can look at collection analytics, cross collection analytics, and concept verticals.

In [ ]:
#figure out how to link other notebooks, especially nice if it's possible to pass the current dataframe